# Foursquare Multiple Models

### Introduction

So in the last lab, we went through the steps of pulling into data to our database using ETL, and then building an api sends back information about particular venues.  In this lesson, we'll work on loading data for multiple models -- like adding a table and model for categories, and associating the proper data.  Let's get started.

### Building the Models

We can begin by adding a model for categories.

> To start describe the relationship between venues and categories.

Now if you look in the models folder, you can see that we already added the `category` and `venue_category` models for you. There are also some updated migrations in the `migrations/create_tables.sql` file.  

> Run the migrations against the `foursquare_test` database.  
> And run the migrations against the `foursquare_development` database.

Then run the tests in `test_category.py`.  Many of the tests should already be passing.

So we've already written a number of initial methods for you.  Still there are a couple that you'll need to add.

* `Category#venues` - add an instance method named `venues` to the category class.  Given a category, it should retrieve it's related venues.

* `Venue#categories` - add an instance method named `categories` to the venues class.  Given a venue, it should retrieve it's related categories.

### Adding to our ETL Functionality

Now that we have our models set up so that we can store venues and their related categories, the next step is to modify our code in the `adapters` folder so that when we hit the foursquare api and pull venues, we also properly relate a venue to it's categories.

### Creating the Category Builder

You can get a sense of how we'll call our `CategoryBuilder` class, by opening up the `manage.py` file (we copied the relevant code below).  There, you'll see that we use the `Client` to `request_venues` from the foursquare api.  Then, for each venue in the list of venues, we separately request the venue, from the api, to get even more data.

Then we pass the response to the `VenueBuilder`, and, when we call `builder.run`, it selects the attributes related to our venues table, creates a Venue instance, and then saves this to the database.

Next, we pass the `venue_response` into the `CategoryBuilder`.  We can see that we also pass the entire `venue_response` into the `CategoryBuilder` and when we call run, we'll need to write code so that it to selects the attributes related to the categories of the venue, and then creates the categories if it does not already exist, and then associates the categories to the venue.

```python
def run():
    venues = []
    client = Client()
    venue_responses = client.request_venues()
    for venue_response in venue_responses:
        venue_response = client.request_venue(venue_response['id'])
        builder = VenueBuilder(venue_response)
        venue = builder.run(conn, cursor)

        builder = CategoryBuilder(venue_response)
        venue = builder.run(conn, cursor)

        venues.append(venue)
    return venues
```

Ok, now let's start building the `CategoryBuilder`, located in the `adapters/category_builder.py` file.  Check your work by passing the tests in the related `tests/adapters/test_category_builder.py` file.  Here are the tasks:

* Update the `CategoryBuilder` so that we can initialize the `CategoryBuilder` with the `venue_reponse`, and it stores the response dictionary as the attribute `venue_response`.  Then confirm that the test passes.

> **Hint:** This is identical to what we did with the `VenueBuilder`, so look to that for assistance.

* Write a `select_attributes` method, that when called, retrieves the category names of the venue.  Pass the related test.

> **Hint**: Access the `venue_response` from the `builder.venue_response` attribute, and then find where the categories are stored by calling `.keys()` on the dictionary.

* Now, if you think about our steps so far, we pass through the entire `venue_response` dictionary, then we use `select_attributes` to return a list of `category_names`.  Next up is to write a method called `find_or_create_categories`, that given a list of category names, returns a list of category instances -- either by finding the related category if it already exists, or by creating a new one if it does not.   

> Hint: Take a look at the `orm.py` file for some useful methods.

* Ok, now so far we have written methods to parse the category names, and find or create the related categories.  But we still will need to associate the categories to a `venue`.  Write a function called `create_venue_categories`, that creates the link between the venue, and it's categories.  That is, it creates the related `venue_category` records.  The function will take in a `venue` instance, and a list of `category` instances, and create the related `venue_categories`.  (Because it is creating new database records, it will also take in argument of `conn` and `cursor`.  Pass the related test.

Finally, if you look at the `CategoryBuilder#run` function, you can see how we tie together the functions we just wrote.   
```python
def run(self, venue_details, venue, conn, cursor):
    category_names = self.select_attributes(venue_details)
    categories = self.find_or_create_categories(category_names, conn, cursor)
    venue_categories = self.create_venue_categories(venue, categories, conn, cursor)
```

### Finishing up

You can see that in our commented out code, we first use a `Builder` object, defined in `adapters/builder.py`.  Take a look to see what this function does.  See if you can follow the code, and place in `breakpoints` to better understand.   

### Updating the Api

Now if we look at what's returned from our flask api, we'll see that our API does not currently return information about our categories.

<img src="./venues-straight.png" width="60%">

Write a `Venue#to_json` function that returns information about the venue and the categories of each venue.  

Then call the `to_json` function in both the `/venues` and `/venues/id` endpoints so that when we make requests to our api, it returns the related categories of a venue. 

<img src="./venue-cats.png" width="70%">

### Summary

In this lesson, we completed our pattern for building both taking in data from an external API with our adapter pattern (ETL).  And of then taking that data from the database returning it when some makes a request to our flask application with the MVC pattern. 

Look through the code, and make sure that you understand the structure of the codebase, and how it works.